In [1]:
import re
import subprocess

## Pylint messages

In [5]:
print(subprocess.check_output(['pylint', '--version']).decode())

pylint 1.7.2, 
astroid 1.5.3
Python 3.6.2 | packaged by conda-forge | (default, Jul 23 2017, 22:59:30) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]



In [6]:
output = subprocess.check_output(
    "pylint --list-msgs | grep '^:' | awk '{print $2}' | cut -c 2-6", 
    shell=True, universal_newlines=True)
pylint_messages = output.split()

In [7]:
len(pylint_messages)

221

## Ignored (.pylinrc) messages

In [8]:
import configparser
config = configparser.ConfigParser()
config.read('../pyta/python_ta/.pylintrc')
pylintrc_messages = [m.strip(",") for m in config['MESSAGES CONTROL']['disable'].split()]

In [9]:
len(pylintrc_messages)

113

## Covered messages

In [30]:
LINK_RE = re.compile('###.*\{#([A-Z][0-9]+)\}.*')  # looks for '{#E0601}'
HEADER_RE = re.compile('###.*\(([A-Z][0-9]+)\).*')  # looks for '### E0601'

In [31]:
link_messages = []
header_messages = []

with open('../website/index.md') as ifh:
    for line in ifh:
        link_messages += LINK_RE.findall(line)
        header_messages += HEADER_RE.findall(line)

In [32]:
assert link_messages == header_messages

In [33]:
covered_messages = link_messages

In [34]:
len(covered_messages)

93

## Missing messages

In [25]:
# Messages in index.md but not in pylint
set(covered_messages) - set(pylint_messages)

{'E0002', 'E9991', 'E9996', 'E9998', 'E9999', 'R0101', 'R0102'}

In [26]:
# Messages in index.md and in .pylintrc
set(covered_messages) & set(pylintrc_messages)

{'E0202', 'E0241', 'E0704', 'W0211'}

In [27]:
# Messages produced by pylint that are not covered by .pylintrc or index.md
missing_messages = set(pylint_messages) - set(pylintrc_messages) - set(covered_messages)

In [28]:
len(missing_messages)

38

In [29]:
for code in missing_messages:
    print(code)
    print(subprocess.check_output("pylint --help-msg={}".format(code), universal_newlines=True, shell=True))

R1707
:trailing-comma-tuple (R1707): *Disallow trailing comma tuple*
  In Python, a tuple is actually created by the comma symbol, not by the
  parentheses. Unfortunately, one can actually create a tuple by misplacing a
  trailing comma, which can lead to potential weird bugs in your code. You
  should always use parentheses explicitly for creating a tuple. This message
  belongs to the refactoring checker. It can't be emitted when using Python <
  3.0.


C0325
:superfluous-parens (C0325): *Unnecessary parens after %r keyword*
  Used when a single item in parentheses follows an if, for, or other keyword.
  This message belongs to the format checker.


C0205
:single-string-used-for-slots (C0205): *Class __slots__ should be a non-string iterable*
  Used when a class __slots__ is a simple string, rather than an iterable. This
  message belongs to the classes checker.


C0330
:bad-continuation (C0330): *Wrong %s indentation%s%s.*
  TODO This message belongs to the format checker.


E0107
:

:fatal (F0001):
  Used when an error occurred preventing the analysis of a module (unable to
  find it for instance). This message belongs to the master checker.


W0301
:unnecessary-semicolon (W0301): *Unnecessary semicolon*
  Used when a statement is ended by a semi-colon (";"), which isn't necessary
  (that's python, not C ;). This message belongs to the format checker.


E0303
:invalid-length-returned (E0303): *__len__ does not return non-negative integer*
  Used when an __len__ method returns something which is not a non-negative
  integer This message belongs to the classes checker.


